# SysOps Associate 
> Notes

- toc: true 
- comments: true
- author: Ankush Agarwal
- categories: [aws, SysOps]

### Overview

![](Intro.png)

### EC2

    Changing Instance Type  - Steps applicable only for EBS Backed Instance
        Need to stop  
        New instance will be assigned
        EBS will get reasssigned
        
#### Enhanced Networking
    EC2 Enhanced Networking (SR-IOV)
        Higher bandwidth, Higher PPS, Lower latency
        Option 1:Elastic Network Adaptor(ENA) upto 100 Gbps
        Option 2:Intel 82599 VF upto 10 Gbps - LEGACY
        Works for newer generation EC2 instances
    Elastic Fabric Adaptor
        Improved ENA for HPC, only works for Linux
        Great for inter-node communications, tightly coupled workloads
        Leverages Message Passing Interface (MPI) standard
        Bypasses the underlying Linux OS to provide low latency, reliable transport
        
#### Placement Group
    Cluster
        is applicable only for specific instances
        Great network - 10 Gbps bandwidth between instances
    Spread
        Limited to 7 instances per AZ per placement group
    Partition
        Upto 7 partitions per AZ
        Upto 100s of EC2 instances
        
#### Shutdown Behaviour using OS
    Terminate Protection
        Only from CLI or Console
        Can enable/disable it on running instances
    Shutdown Behaviour 
        Stop
        Terminate
    
#### EC2 Launch Troubleshooting
    #InstanceLimitExceeded
        Default Running vCPU limit in each region - 64
    #InsuffientInstanceCapacity
        Try again after sometime
    #Instance Terminate Immediately
        Reached EBS Volume Limit
        EBS Snapshot is corrupt
        Do not have permission to access KMS key for decryption
        Instance store backed AMI is missing a required part
        
#### EC2 SSH Trouble
    Possible reason of timeout
        Security Group
        CPU load of the instance is high
        Instance does not have public IP 
        
#### Instance Type
    On Demand
        Linux/Windows - Pay for seconds, other - hour
    Dedicated hosts
        Physical server with a ec2 instance capacity fully dedicated to you
        Allocated for 3 year reservation
        Useful for software that have complicated licensing model - BYOL
    EC2 Dedicated Instances
        Instances running on hardware that's dedicated to you
        May share hardware with other instances in same account
        No control over instance placement
![](DedicatedHosts.png)
    
    
#### Spot Instances & Spot Fleet
    Can choose to stop or terminate your instances within 2 minutes grace period
    First cancel the spot request then cancel the spot instances

#### Instance Type
    R - RAM
    C - Compute
    M - Medium - web app
    I - I/o - Database 
    G - GPU
    T2/T3 burstable instances
    T2/T3 unlimited burst
    
![](InstanceType.png)
    
    
#### Elastic IP
    When you stop then start the instance - public ip changes
    Avoid using Elastic IP
        Use a random public IP & register a DNS name to it
        Use a Load balancer with a static host name
        
#### CloudWatch metrics for EC2
    AWS provided metric
        CPU, Network, Disk(only for instance store) and Status Check Metric
    Custom Metrics
        Basic resolution - 1 minute
        High Resolution - 1 second
        Include RAM, application level metrics      
    Status Check
        Instance Status - Check the EC2 VM
        System Status - Check the underlying hardware        
        
#### Unified CloudWatch Agent
    Push logs & metrics to CloudWatch from EC2 or on-premise server
    procstat plugin
        Collect metrics & monitor system utilization of individual processes
    Download & Install cloudwatch agent in instance
    Run config wizard
    Can put config file in parameter Store
    Can download config file in other ec2 instances
    
#### EC2 Instances Status Check
    System Status Checks
        Stop & Start
    Instance Status Checks
        Reboot
    CloudWatch Metrics
        StatusChedkFailed_System
        StatusCheckFailed_Instance
        StatusCheckFailed(for both)
    Recovery
        CloudWatch Alarm
        Auto Scaling Group
        
    SYSTEM status checks
        System status checks monitor the AWS systems on which your instance runs
        Problem with the underlying host. Example: 
            Loss of network connectivity
            Loss of system power
            Software issues on the physical host
            Hardware issues on the physical host that impact network reachability

        Either wait for AWS to fix the host, OR
        Move the EC2 instance to a new host = STOP & START the instance (if EBS backed)

    INSTANCE status checks
        Instance status checks monitor the software and network configuration of your individual instance
        Example of issues
            Incorrect networking or startup configuration
            Exhausted memory
            Corrupted file system
            Incompatible kernel
            
        Requires your involvement to fix
            Restart the EC2 instance, OR
            Change the EC2 instance configuration     
        
#### EC2 Hibernate
    Stop : Data on disk(EBS) is kept intact
    Terminate : Any EBS volumes(root) also setup to be destroyed is lost
    Hibernate : In-memory state(RAM) is preserved
        Root EBS must be encrypted
        
#### AMI
    AMI built are for a specific region
    AMI are private by default - locked for your account/region
    Stored in S3
    
    AWS Backup Plans to Create AMI
        To maintain Integrity you need to reboot parameter while taking images
        EventBridge + Lambda + CreateImage API with reboot
        
#### EC2 Image Builder
    Used to automate the creation of VM or container Image
    Can be run on schedule

### Systems Manager (SSM) & Opswork

    Helps you manage your EC2 & on-premise systems at scale
    Get operation insight about the state of your infrastructure
    Easily detects problem
    Patching automation for enhanced compliance
    Integrated with CloudWatch metrics/dashboard
    Integrated with AWS Config
    
    Resource Group
    Operations Management
        Explorer
        Ops Center
        CloudWatch Dashboard
        PHD
        Incident Manager
    Shared Resources
        Documents
    Change Management
        Change Manager
        Automation(shut down EC2, create AMIs)
        Change Calendar
        Maintainance Windows
    Application Management
        Application Manager
        AppConfig
        Parameter Store
    Node Management
        Fleet Manager
        Compliance
        Inventory:Discover & Audit software installed
        Hybrid Activations
        Session Manager
        Run command
        State Manager:Define & maintaining configuration
        Patch Manager
        Distributer        
    
#### How SSM Works
    We need to install SSM Agents onto the systems we control
    Installed by default on Linux AMI & some Ubuntu AMI
    If an instance can't be controlled with SSM, probbaly an issue wih SSM Agent
    Make sure that EC2 instance have proper IAM role to allow SSM actions

#### Start EC2 instances with SSM Agent
    Just add IAM role which allow connections to SSM, it will start showing in SSM Manage Instances
    
#### AWS Tags & SSM Resource Groups
    Create, view or manage logical group of resources thanks to tags
    Allow creation of logical group of resources such as 
        Applications
        Different layers of an application stack
        Production vs development environments
    Regional Service
    Works with EC2,S3,DynamoDB,Lambda
    
#### SSM Documents & SSM Run Command
    Documents
        Can be in JSON or YAML
        You define parameters
        You define actions
        Many documents already exist in AWS
        Can be applied to
            State Manager
            Patch Manager
            Automation
            Run command
            Reference Parameter Store
            
    Run Commands
        Execute a document(script) or just run a command
        Run command across multiple instance using resource groups
        Rate Control/Error Control
        Integrated with IAM & CloudTrail
        No need for SSH
        Results in the console or S3 or cloudwatch
        
#### SSM - Automation
    Simplify common maintenance & deployment tasks of EC2 instances & other AWS resources
    Example - Restart Instances, Create an AMI, EBS Snapshots
    Automation Runbook
        SSM Documents of Type Automation
        Define actions performed on your EC2 instances or AWS resources
        Pre-defined runbooks or create custom runbooks
    Can be Triggered
        AWS Console, CLI, SDK
        Event Bridge
        On a schedule using Maintenance Window
        By AWS Config for rules remediation

#### AWS Parameter Store
    Secure storage for configurations & secret
    Optional seamless encryption using KMS
    Serverless, scalable, durable, easy SDK, free
    Version tracking of configurations/secrets
    Configurations management using path & IAM
    Notifications with CloudWatch Events
    Integration with CloudFormation
        
#### SSM Inventory & Patches
    Inventory - List Software on an Instance
    Inventory + Run Command - Patch Software
    Patch Manager + Maintenance Window - Patch OS
    Patch Manager - Gives you Compliance
    State Manager - Ensures instance are in a consistence state(compliance)
    
#### SSM Secure Shell
    Session Manager allows you to start a secure shell on your VM
    Does not use SSH access & bastions hosts
    Only EC2 now
    Log actions done through secure shells to S3 & CloudWatch Logs
    IAM Permission:Access SSM + Write to S3 + write to CloudWatch
    CloudTrail can intercept StartSession events
    AWS Secure Shell versus SSH
        No need to open the port 22 at all anymore (Security)
        No need for bastions hosts
        All commands are logged to S3/CloudWatch (auditing)
        Access to Secure Shell is done through User IAM,not SSH keys
            
#### Opsworks
    Chef or Puppet help you perform server configurations automatically or repetitive actions  

### High Availablity & Scalability

#### Load Balancer Overview
    Expose a single point of DNS to your application
    Seamlessly handle failure of downstream instances
    Do regulare health check to your instances
    Provide SSL Termination(HTTPS) for your websites -- CLB & ALB
    Enforce stickiness with cookies
    Can setup private or public Load Balancer
    Load Balancer Errors 503 means at capacity or no registered target
    If LB can't connect to your application, check your Security Groups
    
    Types
        Classic - 2009
        Application - 2016
        Network - 2017

#### Application Load Balancer
    ALB (Layer 7) allows to do:
        Load balancing to multiple HTTP applications across machine(target groups)
        Load balancing to multiple applications on the same machine(containers)
        Load balancing based on route in URL
        Load balancing based on hostname in URL
        
    Are used for microservices & container based application (Docker & Amazon ECS)
    Has a port mapping feature to redirect to a dynamic url
    
    Stickiness can be enables at the target group level
        Same request goes to the same instance
        Stickiness is diretly generated by the ALB
        
    ALB supports HTTP/HTTPS & WebSockets protocols
    
#### Network Load Balancer
    NLB (Layer 4) allows to do:
        Forward TCP traffic to your instances
        Handles million of requests per seconds
        Support for static IP or elastic IP
        Less latency ~ 100 ms (vs 400 ms for ALB)
    Can directly see the client IP
        
#### Metrics, Logging & Tracing for ELB
    All LB metrics are directly pushed to CloudWatch metrics
        BackendConnectionErrors
        HealthyHostCount / UnHealthyHostCount
        HTTPCode_Backend_2XX:Successful Request
        HTTPCode_Backend_3XX:Redirect Request
        HTTPCode_ELB_4XX:Client Error
        HTTPCode_ELB_5XX:Server Error
        Latency
        RequestCount
        SurgeQueueLength
        SpillOverLength
    Access Log can be stored in S3
    
#### Auto Scaling Group
    A launch configuration
        AMI + Instance Type
        EC2 User Data
        EBS volumes
        SG
        SSH Key Pair
    Min Size/Max Size/Initial Capacity
    Network + Subnets Information
    Load Balancer Information
    Scaling Policies
    
    Auto Scaling Alarms
        Cloudwatch alarms
        EC2 managed rules
        Custom metrics
        
    AZ Rebalance
        Launch new instance then terminate old instance
        
        If you suspend the Launch process
            AZ won't launch instance 
            AZ won't terminate instance 
            
        If you suspend the Terminate process
            AZ can grow upto 10% of its size
            AZ will remian at this capacity since it can't terminate instance

### Elastic Beanstalk
    Manged Service
    Architecture Models
        Single Instance Deployment: Good for Dev
        LB + ASG: Good for Prod or Non-Prod
        ASG only: Good for non-web apps in production (workers,etc..)
        
    Three components
        Application
        Application version
        Environment name
    
    